# This is an EDA challenge for SXOPE interview process for Data Analyst position

In [1]:
#Memory cleansing
import gc
gc.collect()

7

In [2]:
#Importing packages
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

import plotly.express as px


C:\Users\Sasha Evans\AppData\Roaming\Python\Python37\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#Reading data: claims_sample_data.xlsx

my_sheet = 'Sheet1' # sheet_name
file_name = 'claims_sample_data.xlsx' # name of your excel file
df = pd.ExcelFile(file_name).parse(my_sheet) # importing data into DataFrame object

print(df.head())

    MONTH SERVICE_CATEGORY CLAIM_SPECIALTY    PAYER  PAID_AMOUNT
0  201801     AncillaryFFS             NaN  Payer F         4281
1  201801     AncillaryFFS             NaN  Payer H         2221
2  201801     AncillaryFFS             NaN  Payer O         3937
3  201801     AncillaryFFS             NaN  Payer W          268
4  201801     AncillaryFFS             ACH  Payer W          151


This dataset is a sampled aggregated data for the period of 2018/01 - 2020/07 (numbers are fictional). 
The dataset contains the following columns:

    •	MONTH - a month claims were lodged
    •	SERVICE_CATEGORY - a department that provided services to patients
    •	CLAIM_SPECIALTY - a type of medical services by an official classification system
    •	PAYER - an insurance company
    •	PAID_AMOUNT - sum of expenses (claims), $

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52152 entries, 0 to 52151
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   MONTH             52152 non-null  int64 
 1   SERVICE_CATEGORY  52152 non-null  object
 2   CLAIM_SPECIALTY   51901 non-null  object
 3   PAYER             52152 non-null  object
 4   PAID_AMOUNT       52152 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


In [5]:
df.nunique()

MONTH                 32
SERVICE_CATEGORY      10
CLAIM_SPECIALTY      905
PAYER                 10
PAID_AMOUNT         7946
dtype: int64

In [6]:
df.isna().sum()

MONTH                 0
SERVICE_CATEGORY      0
CLAIM_SPECIALTY     251
PAYER                 0
PAID_AMOUNT           0
dtype: int64

In [7]:
print('CLAIM_SPECIALTY missing values ', round(251/52152*100,2), '%')

CLAIM_SPECIALTY missing values  0.48 %


In [8]:
df['CLAIM_SPECIALTY'].value_counts()

INTERNAL MEDICINE                    1227
FAMILY PRACTICE                      1128
GENERAL PRACTICE                      949
GENERAL SURGERY                       806
PATHOLOGY                             772
                                     ... 
Physical Therapist - Pediatrics\t       1
Adolescent Medicine                     1
Surgery, Oncologic                      1
Pediatrics - Cardiology                 1
SUORSP                                  1
Name: CLAIM_SPECIALTY, Length: 905, dtype: int64

In [ ]:
fig = px.histogram(df, x="CLAIM_SPECIALTY")
fig.show()

From the graphics we can see that the distribution of CLAIM_SPECIALTY is scewed to the left and some values occur much less
than the others. Since the missing values are less than 0.5% there is no harm in deleting those records for model training.

In [10]:
df['CLAIM_SPECIALTY'].describe()

count                 51901
unique                  905
top       INTERNAL MEDICINE
freq                   1227
Name: CLAIM_SPECIALTY, dtype: object

In [11]:
df=df.dropna()

In [12]:
#Checking for duplicated records
duplicate_rows = df[df.duplicated()]
duplicate_rows.shape

(0, 5)

In [13]:
#Formatting the data
df['MONTH_str'] = df['MONTH'].astype(str)
df.head(5)

,MONTH,SERVICE_CATEGORY,CLAIM_SPECIALTY,PAYER,PAID_AMOUNT,MONTH_str
4,201801,AncillaryFFS,ACH,Payer W,151,201801
5,201801,AncillaryFFS,Advanced Registered Nurse Prac,Payer UN,0,201801
6,201801,AncillaryFFS,ADVANCED RN PRACT,Payer CA,443,201801
7,201801,AncillaryFFS,AMBULANCE,Payer CA,14890,201801
8,201801,AncillaryFFS,AMBULANCE,Payer CO,1722,201801


In [14]:
df['MONTH_dt'] = pd.to_datetime(df['MONTH_str'], format='%Y%m', errors='coerce')
df.head(5)

,MONTH,SERVICE_CATEGORY,CLAIM_SPECIALTY,PAYER,PAID_AMOUNT,MONTH_str,MONTH_dt
4,201801,AncillaryFFS,ACH,Payer W,151,201801,2018-01-01
5,201801,AncillaryFFS,Advanced Registered Nurse Prac,Payer UN,0,201801,2018-01-01
6,201801,AncillaryFFS,ADVANCED RN PRACT,Payer CA,443,201801,2018-01-01
7,201801,AncillaryFFS,AMBULANCE,Payer CA,14890,201801,2018-01-01
8,201801,AncillaryFFS,AMBULANCE,Payer CO,1722,201801,2018-01-01


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51901 entries, 4 to 52151
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   MONTH             51901 non-null  int64         
 1   SERVICE_CATEGORY  51901 non-null  object        
 2   CLAIM_SPECIALTY   51901 non-null  object        
 3   PAYER             51901 non-null  object        
 4   PAID_AMOUNT       51901 non-null  int64         
 5   MONTH_str         51901 non-null  object        
 6   MONTH_dt          51890 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 3.2+ MB


In [16]:
df=df.drop(['MONTH_str', 'MONTH'], axis=1)

In [17]:
df.rename(columns={'MONTH_dt': 'MONTH'}, inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51901 entries, 4 to 52151
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   SERVICE_CATEGORY  51901 non-null  object        
 1   CLAIM_SPECIALTY   51901 non-null  object        
 2   PAYER             51901 non-null  object        
 3   PAID_AMOUNT       51901 non-null  int64         
 4   MONTH             51890 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.4+ MB


In [ ]:
print(df['MONTH'].describe())
fig = px.histogram(df, x="MONTH")
fig.show()

From this distribution we can see that the amount of claims started to drop from April 2020, which falls on the start of the 
COVID-19 pandemic. From my research on the explanation of this:
    "In spring of 2020, healthcare use and spending dropped precipitously due to cancellations of elective care to increase 
    hospital capacity and social distancing measures to mitigate community spread of the coronavirus. Although telemedicine 
    use increased sharply, it was not enough to compensate for the drop in in-person care. As the year progressed, healthcare 
    use and spending began to rebound as in-person care resumed for hospital and lab services and COVID-19 testing became 
    more widely available. However, overall health spending appears to have dropped slightly in 2020, the first time in 
    recorded history."
(c)https://www.healthsystemtracker.org/chart-collection/how-have-healthcare-utilization-and-spending-changed-so-far-during-the-coronavirus-pandemic/

Let's look at variables and their distributions

In [ ]:
print(df['PAID_AMOUNT'].describe())
plt.figure(figsize=(9, 8))
sns.distplot(df['PAID_AMOUNT'], color='g', bins=100, hist_kws={'alpha': 0.4});

From this graphics we can see that the claim's amounts are scewed to the left and have one strongly prevailing value - 0. 
It is also unusual that there are some negative values for claimed amount.

In [14]:
#Removing outliers for PAID_AMOUNT
Q1 = df['PAID_AMOUNT'].quantile(0.25)
Q3 = df['PAID_AMOUNT'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

1265.0


In [15]:
df = df[~((df['PAID_AMOUNT'] < (Q1 - 1.5 * IQR)) |(df['PAID_AMOUNT'] > (Q3 + 1.5 * IQR)))]
df.shape

(46616, 6)

In [ ]:
plt.figure(figsize=(9, 8))
sns.distplot(df['PAID_AMOUNT'], color='g', bins=100, hist_kws={'alpha': 0.4});

Now we see the distribution became better. 
There are records with negative values for PAID_AMOUNT.
This situation can happen when:
1. When a beneficiary is charged the full Insurance company deductible during a short inpatient stay and the deductible exceeds the amount insurance company pays; or,
2. When a beneficiary is charged a coinsurance amount during a long stay and the coinsurance amount plus deductible exceeds the amount insurance company pays
So these values are not errors but correct entries in the data with a different meaning. They do not mean the amount has been deducted from the beneficiary.

In [ ]:
print(df['SERVICE_CATEGORY'].describe())
fig = px.histogram(df, x="SERVICE_CATEGORY")
fig.show()

In [ ]:
print(df['PAYER'].describe())
fig = px.histogram(df, x="PAYER")
fig.show()

In [ ]:
#Let's investigate amount claimed in slice by other variables' values.

In [ ]:
fig = px.histogram(df, x="MONTH", y="PAID_AMOUNT")
fig.show()

In [ ]:
fig = px.histogram(df, x="SERVICE_CATEGORY", y="PAID_AMOUNT")
fig.show()

In [28]:
fig = px.histogram(df, x="CLAIM_SPECIALTY", y="PAID_AMOUNT")
fig.show()

In [16]:
#Calculating average claim
df['claim_count']=1
df_avg_claim = df.groupby(['CLAIM_SPECIALTY']).sum().reset_index()
df_avg_claim.head(5)

,CLAIM_SPECIALTY,MONTH,PAID_AMOUNT,claim_count
0,ABULATORY SURGICAL CENTER,4440832,36337,22
1,ACCUP,201812,147,1
2,ACH,807526,917,4
3,ACUPU,403817,602,2
4,ACUPUNCTURIST,201906,423,1


In [17]:
df_avg_claim['AVG_PAID_AMOUNT'] = round(df_avg_claim['PAID_AMOUNT']/df_avg_claim['claim_count'],2)
df_avg_claim = df_avg_claim.sort_values('AVG_PAID_AMOUNT', ascending=False)
df_avg_claim.head(5)

,CLAIM_SPECIALTY,MONTH,PAID_AMOUNT,claim_count,AVG_PAID_AMOUNT
779,SNF,202006,3175,1,3175.0
739,RENAL DIALYSIS FACILITY,202006,3019,1,3019.0
732,REHAB HOSP,1412930,17101,7,2443.0
488,OTHERS/SWINGBED,403717,4464,2,2232.0
737,REHABILITATION UNIT,202003,2015,1,2015.0


In [18]:
#I will select top 20 most popular claims
df_avg_claim.nlargest(20,'claim_count').sort_values('claim_count', ascending=False)

,CLAIM_SPECIALTY,MONTH,PAID_AMOUNT,claim_count,AVG_PAID_AMOUNT
215,FAMILY PRACTICE,199463380,899765,988,910.69
313,INTERNAL MEDICINE,197845988,920696,980,939.49
228,GENERAL PRACTICE,178869820,627249,886,707.96
534,PATHOLOGY,148182300,674403,734,918.81
230,GENERAL SURGERY,140913561,686029,698,982.85
581,PODIATRY,122946227,397518,609,652.74
226,GASTROENTEROLOGY,121130509,694203,600,1157.01
722,RADIOLOGY,119111265,648880,590,1099.80
73,CARDIOLOGY,112248359,513895,556,924.27
29,ANESTHESIOLOGY,111843018,620753,554,1120.49


In [ ]:
#Most popular claims sorted by the average size of claim
fig=px.bar(df_avg_claim.nlargest(20,'claim_count').sort_values('AVG_PAID_AMOUNT', ascending=True), x='AVG_PAID_AMOUNT', y='CLAIM_SPECIALTY', orientation='h', text_auto=True)
fig.show()

Projection for next 6 months: due to the COVIV-19 pandemic the changes in trends are not predictable with the data, because
the data analysis normally predicts trends that are in line with the common situation. It projects historical trends into
the future. In case of major changes in the world and force-majeur situations, it requires completely different approach.
I had to perform extra research to find the following projections on the payments to the patients:
    "Spending in Q3, while more alike the historical 2019 trend than Q2 spending, has not rebounded to equalize the Q2 boon. 
    With potential impending lockdowns and the threat of exceeding hospital capacity as COVID numbers crest in Q4, the final 
    quarter of 2020 will potentially look more like Q2 than its 2019 counterpart. "
Source:
    https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7869966/